<a href="https://colab.research.google.com/github/csalnav2/PhotonMechanic/blob/main/LLMnodeprobe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok --quiet # Install the pyngrok module
from pyngrok import ngrok

In [ ]:
!ngrok config add-authtoken 2tedizGJDmEJMDtfhCR6ftx8EyX_2RaA7t8nPn17SU7jiyK1H

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!kill -9 $(lsof -t -i:8000) 2>/dev/null || echo "No existing process"


No existing process


In [ ]:
!pip install --quiet flask transformers torch networkx matplotlib scikit-learn safetensors pyvis soundfile psutil GPUtil
# no dynamic quant error on GPU for int8 => fallback to CPU

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.7 MB/s eta 0:00:00


In [ ]:
!pkill -f ngrok

In [7]:
# =============================================
# Single Cell or local Python script
#   - Kills old processes on port=8000
#   - Installs dependencies
#   - Writes interpret_flask.py
#   - Runs Flask in background
#   - Polls for readiness or times out
#   - Then starts ngrok
# =============================================

import os, time, requests

print("🔴 Killing anything on port=8000 + old ngrok sessions.")
try:
    get_ipython().system("pkill -f ngrok 2>/dev/null || echo 'No existing ngrok process.'")
    get_ipython().system("kill -9 $(lsof -t -i:8000) 2>/dev/null || echo 'No existing process.'")
except:
    pass

print("📦 Installing dependencies (quietly)...")
try:
    get_ipython().system("pip install --quiet flask transformers torch networkx matplotlib pyvis psutil requests pyngrok scipy GPUtil plotly")
except:
    pass

FULL_CODE = r'''
###############################################################################
# interpret_flask.py
#
# AI Interpretability web app with:
#   - BFS layering
#   - Shape-based memory estimates
#   - More realistic time complexities based on node type + user-specified seq_len
#   - Continuous drag => set target_mem/time
#   - Automatic background convergence of mem/time
#   - Real-time polling via /poll_compgraph_data => updates tooltips every 3s
#   - BFS=0 layer route shows only BFS=0 input nodes
###############################################################################

import os
import time
import random
import threading
import traceback
import torch
import torch.nn as nn
import torch.fx as fx
import networkx as nx
import psutil
import GPUtil
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from io import BytesIO
from flask import Flask, request, send_file, jsonify, Response

from transformers import AutoModel, AutoTokenizer
from pyvis.network import Network
from scipy.interpolate import griddata
import csv
import json
import requests

app = Flask(__name__, static_folder="static")
if not os.path.exists("static"):
    os.makedirs("static")

################################################
# GLOBALS
################################################

RL_CHOICE = "No RL"
HYP_GRAPH = None
COMP_GRAPH = None
OLD_HYP_GRAPH = None
OLD_COMP_GRAPH = None

MODEL_CACHE = {}
CONVERSATION = []
COMP_SHAPE_INFO = {}  # { "COMP_nodeName": (in_shape, out_shape) }

# Q_TABLE => 10 states, each with 4 actions
Q_TABLE = {s:[random.uniform(-1,1) for _ in range(4)] for s in range(10)}

GPU_MEMORY_SCALE = {
    "T4":1.0,
    "A100":0.5,
    "V100":0.7,
    "TPU":0.4
}

AVAILABLE_MODELS = {
    "BERT (base)":"bert-base-uncased",
    "GPT-2":"gpt2",
    "DistilBERT":"distilbert-base-uncased"
}
AVAILABLE_GPUS = ["T4","A100","V100","TPU"]
QUANT_METHODS = ["None (FP32)","FP16","INT8"]
REASONING_METHODS = [
    "No reasoning",
    "Monte Carlo Tree Search (MCTS)",
    "Chain of Thought (CoT)",
    "Meta-CoT",
    "Neural-Symbolic"
]
RL_ALGORITHMS = [
    "No RL",
    "Q-Learning",
    "Deep Q-Network (DQN)",
    "Policy Gradient",
    "Actor-Critic",
    "PPO"
]

###############################################################################
# TORCH FX SHAPE TRACING
###############################################################################

def trace_model_fx(model, tokenizer):
    device = next(model.parameters()).device
    wrap = ModelWrapper(model, tokenizer).to(device)
    enc = tokenizer("Hello world!", return_tensors="pt").to(device)

    shape_info = {}
    try:
        traced = fx.symbolic_trace(
            wrap,
            concrete_args={
                "input_ids": enc["input_ids"],
                "attention_mask": enc.get("attention_mask", None)
            }
        )
        from torch.fx.passes.shape_prop import ShapeProp
        sp = ShapeProp(traced)
        sp.run(enc)

        for node in traced.graph.nodes:
            node_name = str(node)
            in_shape  = "?"
            out_shape = "?"
            try:
                if "tensor_meta" in node.meta:
                    out_shape = tuple(node.meta["tensor_meta"].shape)
            except:
                pass
            shape_info[node_name] = (in_shape, out_shape)
    except:
        pass

    return shape_info

def estimate_gpu_memory(node_name, shape_tuple):
    """Rough memory estimate in KB for the output shape."""
    if not isinstance(shape_tuple, tuple):
        return "Unknown"
    numel = 1
    for dim in shape_tuple:
        numel *= dim
    mem_kb = (numel * 4) / 1024  # FP32 = 4 bytes
    return f"{mem_kb:.1f}"

def interpret_operation(node_name):
    """Heuristic parse of node_name -> operation type (for table in glossary)."""
    low = node_name.lower()
    if "matmul" in low:
        return "MatrixMultiplication"
    elif "transpose" in low:
        return "Transpose"
    elif "softmax" in low:
        return "Softmax"
    elif "norm" in low:
        return "LayerNorm"
    elif "activation" in low or "gelu" in low or "relu" in low:
        return "Activation"
    elif "linear" in low or "dense" in low:
        return "Linear"
    elif "dropout" in low:
        return "Dropout"
    elif "pooler" in low or "pool" in low:
        return "Pooling"
    elif "attention" in low:
        return "Attention block"
    elif "embedding" in low:
        return "Embedding"
    return "Generic op"

###############################################################################
# MODEL UTILS
###############################################################################

def load_model(hf_id, quant="None (FP32)"):
    import torch.nn as nn
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if quant=="INT8" and device=="cuda":
        device="cpu"

    cache_key = f"{hf_id}_{quant}_{device}"
    if cache_key in MODEL_CACHE:
        return MODEL_CACHE[cache_key]["model"], MODEL_CACHE[cache_key]["tokenizer"]

    model = AutoModel.from_pretrained(hf_id)
    tokenizer = AutoTokenizer.from_pretrained(hf_id)
    if quant=="FP16":
        model = model.half()
    elif quant=="INT8":
        model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

    model.eval()
    if quant!="INT8":
        model.to(device)

    MODEL_CACHE[cache_key] = {"model": model, "tokenizer": tokenizer}
    return model, tokenizer

###############################################################################
# APPROX TIME COMPLEXITY
###############################################################################
def approximate_time_complexity(node_str, reasoning, seq_len):
    """
    A toy function that tries to reflect different complexities.
    - If "attention" or "matmul" => O(n^2)
    - If "embedding", "norm", "linear", "pool" => O(n)
    - If "sparseattn" => O(n log n) as a possible example
    - Then we also scale by reasoning method
    """
    low = node_str.lower()

    # Base complexity factor from node type
    # We'll just treat them as integer "costs" for demonstration.
    base = 1
    if "attention" in low or "matmul" in low:
        # e.g. O(n^2) => we do seq_len^2
        base = seq_len*seq_len
    elif "sparse" in low and "attn" in low:
        # e.g. O(n log n)
        import math
        base = int(seq_len * np.log2(seq_len+1))
    elif "embedding" in low or "norm" in low or "linear" in low or "pool" in low:
        # O(n)
        base = seq_len
    else:
        # Just a constant
        base = 1

    # Reasoning multiplier
    # This is totally arbitrary, just to illustrate synergy with "reasoning" field
    mul = 1
    if reasoning=="Monte Carlo Tree Search (MCTS)":
        mul = 2
    elif reasoning=="Chain of Thought (CoT)":
        mul = 3
    elif reasoning=="Meta-CoT":
        mul = 4
    elif reasoning=="Neural-Symbolic":
        mul = 5

    return base * mul

def toy_finetune(model):
    device = next(model.parameters()).device
    model.train()
    x = torch.randint(0,1000,(2,8), device=device)
    m = torch.ones_like(x)
    out = model(input_ids=x, attention_mask=m)
    if isinstance(out, tuple) and out and isinstance(out[0], torch.Tensor):
        loss = out[0].sum()
    elif hasattr(out, "last_hidden_state"):
        loss = out.last_hidden_state.sum()
    else:
        loss = torch.tensor(0.0, device=device)
    optim = torch.optim.AdamW(model.parameters(), lr=1e-5)
    optim.zero_grad()
    loss.backward()
    optim.step()
    model.eval()

class ModelWrapper(nn.Module):
    def __init__(self, submodel, tokenizer):
        super().__init__()
        self.submodel = submodel
        self.tokenizer = tokenizer
    def forward(self, input_ids, attention_mask=None):
        return self.submodel(input_ids=input_ids, attention_mask=attention_mask)

###############################################################################
# HYPERGRAPH
###############################################################################

def extract_hypergraph(model, quant, reasoning):
    import networkx as nx
    g = nx.Graph()
    for name, module in model.named_modules():
        node_name = "HYP_" + name
        g.add_node(node_name)
        if "." in name:
            parent = "HYP_" + name.rsplit(".",1)[0]
            child  = "HYP_" + name
            g.add_edge(parent, child)
    return g

###############################################################################
# COMP GRAPH BFS
###############################################################################

def fallback_comp_graph_for_gpt2(model, reasoning, gpu_sel):
    import networkx as nx
    scale_factor = GPU_MEMORY_SCALE.get(gpu_sel,1.0)
    g = nx.Graph()
    for name, module in model.named_modules():
        node_name = "COMP_" + name
        g.add_node(node_name,
                   BFS_distance=0,
                   mem=0,
                   target_mem=0,
                   timecost=0,
                   target_time=0)
        if "." in name:
            parent = "COMP_" + name.rsplit(".",1)[0]
            child  = "COMP_" + name
            g.add_edge(parent, child)
    return g

def extract_comp_graph(model, tokenizer, gpu_sel, quant, reasoning):
    import networkx as nx
    import torch.fx as fx
    device = next(model.parameters()).device
    wrap = ModelWrapper(model, tokenizer).to(device)
    enc  = tokenizer("Hello world!", return_tensors="pt").to(device)

    try:
        traced = fx.symbolic_trace(
            wrap,
            concrete_args={
                "input_ids": enc["input_ids"],
                "attention_mask": enc.get("attention_mask", None)
            }
        )
        g = nx.Graph()
        for node in traced.graph.nodes:
            node_name = str(node)
            nm = "COMP_" + node_name
            g.add_node(nm,
                       BFS_distance=0,
                       mem=0,
                       target_mem=0,
                       timecost=0,
                       target_time=0)
        for node in traced.graph.nodes:
            for user in node.users:
                g.add_edge("COMP_"+str(node), "COMP_"+str(user))
        return g
    except:
        return fallback_comp_graph_for_gpt2(model, reasoning, gpu_sel)

def compute_bfs_levels(nx_graph):
    import networkx as nx
    if nx_graph.number_of_nodes()==0:
        return
    root_candidates = [n for n in nx_graph.nodes if "input" in n.lower()]
    if not root_candidates:
        in_degs = nx_graph.in_degree()
        zero_in = [nd for (nd,deg) in in_degs if deg==0]
        if zero_in:
            root_candidates= zero_in
        else:
            root_candidates= [list(nx_graph.nodes)[0]]
    visited= set()
    queue= []
    for r in root_candidates:
        queue.append((r,0))
        nx_graph.nodes[r]["BFS_distance"]=0
        visited.add(r)
    while queue:
        current, dist= queue.pop(0)
        for neighbor in nx_graph.neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                nx_graph.nodes[neighbor]["BFS_distance"]= dist+1
                queue.append((neighbor, dist+1))

def get_layer_subgraph(nx_graph, layer_num):
    import networkx as nx
    subg= nx.Graph()
    if nx_graph.number_of_nodes()==0:
        return subg
    if "BFS_distance" not in list(nx_graph.nodes(data=True))[0][1]:
        compute_bfs_levels(nx_graph)

    # BFS=0 => only BFS=0 nodes
    if layer_num == 0:
        for n,d in nx_graph.nodes(data=True):
            if d.get("BFS_distance",None)==0:
                subg.add_node(n,**d)
        for (u,v) in nx_graph.edges():
            if u in subg.nodes and v in subg.nodes:
                subg.add_edge(u,v,**nx_graph[u][v])
        return subg

    for n,d in nx_graph.nodes(data=True):
        if d.get("BFS_distance",None)==layer_num:
            subg.add_node(n,**d)
    for (u,v) in nx_graph.edges():
        if (u in subg.nodes) and (v in subg.nodes):
            subg.add_edge(u,v,**nx_graph[u][v])
    return subg

################################################
# Q-TABLE & RL
################################################

def update_q_table(state, action, reward):
    old_val= Q_TABLE[state][action]
    new_val= old_val + 0.1*(reward - old_val)
    Q_TABLE[state][action]= new_val

################################################
# 2D & 3D Q-SPACE
################################################

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import plotly.graph_objects as go

@app.route("/q_space")
def q_space():
    states = sorted(Q_TABLE.keys())
    xs,ys,zs= [],[],[]

    for si,s in enumerate(states):
        for ai,val in enumerate(Q_TABLE[s]):
            xs.append(si)
            ys.append(ai)
            zs.append(val)
    import numpy as np
    xs= np.array(xs)
    ys= np.array(ys)
    zs= np.array(zs)

    xi= np.linspace(xs.min(), xs.max(), 50)
    yi= np.linspace(ys.min(), ys.max(), 50)
    Xi,Yi= np.meshgrid(xi,yi)
    from scipy.interpolate import griddata
    Zi= griddata((xs,ys),zs,(Xi,Yi), method="cubic")

    fig, ax = plt.subplots(figsize=(4,4))
    cont= ax.contourf(Xi,Yi,Zi,10,cmap="coolwarm")
    fig.colorbar(cont,ax=ax)
    ax.set_xlabel("State")
    ax.set_ylabel("Action")
    ax.set_title("Q-Space Heatmap (2D)")
    ax.scatter(xs,ys,c="black",s=15,marker="x")

    buf= BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    plt.close(fig)
    return send_file(buf, mimetype="image/png")

def polynomial_fit_r2(xs, ys, zs):
    import numpy as np
    X= np.column_stack((np.ones_like(xs), xs, ys, xs*xs, xs*ys, ys*ys))
    coeffs,residuals,rank,s= np.linalg.lstsq(X,zs,rcond=None)
    z_pred= X @ coeffs
    ss_res= np.sum((zs - z_pred)**2)
    ss_tot= np.sum((zs - np.mean(zs))**2)
    if ss_tot<1e-12:
        return 1.0
    r2=1-(ss_res/ss_tot)
    return r2

@app.route("/q_space_interactive")
def q_space_interactive():
    import numpy as np
    states= sorted(Q_TABLE.keys())
    xs,ys,zs= [],[],[]
    for si,s in enumerate(states):
        for ai,val in enumerate(Q_TABLE[s]):
            xs.append(float(si))
            ys.append(float(ai))
            zs.append(val)
    xs= np.array(xs)
    ys= np.array(ys)
    zs= np.array(zs)

    minZ,maxZ= zs.min(), zs.max()
    if abs(maxZ-minZ)<1e-9:
        z_scaled= np.full_like(zs,4.5)
    else:
        z_scaled= 9.0*(zs-minZ)/(maxZ-minZ)

    r2_value= polynomial_fit_r2(xs,ys,z_scaled)

    Xi= np.linspace(0,9,10)
    Yi= np.linspace(0,9,10)
    XiGrid,YiGrid= np.meshgrid(Xi,Yi)
    from scipy.interpolate import griddata
    Zi= griddata((xs,ys),z_scaled,(XiGrid,YiGrid), method="cubic")
    if np.isnan(Zi).any():
        Zi2= griddata((xs,ys),z_scaled,(XiGrid,YiGrid), method="nearest")
        Zi[np.isnan(Zi)] = Zi2[np.isnan(Zi)]

    fig= go.Figure(data=[go.Surface(z=Zi,x=Xi,y=Yi,colorscale="Viridis")])
    fig.update_traces(contours_z=dict(show=True,usecolormap=True,highlightcolor="limegreen",project_z=True))
    fig.update_layout(
        title=f"3D Q-Space R²={r2_value:.4f}",
        autosize=False,
        width=700,
        height=600,
        margin=dict(l=65,r=50,b=65,t=90),
        scene=dict(
            xaxis=dict(title="State", range=[0,9],autorange=False),
            yaxis=dict(title="Action", range=[0,9],autorange=False),
            zaxis=dict(title="Q-Value", range=[0,9],autorange=False),
            camera=dict(eye=dict(x=1.6,y=1.6,z=1.2)),
            aspectmode="cube"
        )
    )
    fig.update_layout(scene_dragmode="orbit")

    fig.write_html("static/qspace_3d.html")
    return f"""
<h2>3D Q-Space</h2>
<p>R²={r2_value:.4f}</p>
<iframe src="/static/qspace_3d.html" width="700" height="600"></iframe>
"""

###############################################################################
# SUBGRAPH + GLOSSARY
###############################################################################

def build_subgraph_hyper(original_graph, filter_str):
    import networkx as nx
    fl= filter_str.lower()
    subg= nx.Graph()
    for n,d in original_graph.nodes(data=True):
        if fl in n.lower():
            subg.add_node(n,**d)
    for (u,v) in original_graph.edges():
        if (u in subg.nodes) and (v in subg.nodes):
            subg.add_edge(u,v,**original_graph[u][v])
    return subg

@app.route("/layer_hypergraph/<filter_str>")
def layer_hypergraph(filter_str):
    global HYP_GRAPH
    if not HYP_GRAPH:
        return "<h3>No HYP_GRAPH. Run first!</h3>"
    subg= build_subgraph_hyper(HYP_GRAPH, filter_str)
    if subg.number_of_nodes()==0:
        return f"<h3>No hypergraph nodes match filter={filter_str}</h3>"

    net_h= Network(height="500px", width="600px", directed=False)
    net_h.repulsion(node_distance=140, central_gravity=0.1)
    for n,d in subg.nodes(data=True):
        net_h.add_node(n, label=n, shape="dot", size=20, borderWidth=1,
                       title=n, color={"border":"black","background":"blue"})
    for (u,v) in subg.edges():
        net_h.add_edge(u,v, color="gray")

    net_h.write_html("static/hypergraph_layer.html", notebook=False, open_browser=False)
    return f'<h2>Hypergraph filter={filter_str}</h2><iframe src="/static/hypergraph_layer.html" width="600" height="500"></iframe>'

@app.route("/layer_compgraph/<layer_str>")
def layer_compgraph(layer_str):
    global COMP_GRAPH
    if not COMP_GRAPH:
        return "<h3>No COMP_GRAPH. Run first!</h3>"
    try:
        layer_num= int(layer_str)
    except:
        return f"<h3>Invalid layer number: {layer_str}</h3>"

    subg= get_layer_subgraph(COMP_GRAPH, layer_num)
    if subg.number_of_nodes()==0:
        return f"<h3>No compgraph BFS_distance={layer_num}</h3>"

    net_c= Network(height="500px", width="600px", directed=False)
    net_c.repulsion(node_distance=140, central_gravity=0.1)
    for n,d in subg.nodes(data=True):
        BFS_d= d.get("BFS_distance",0)
        mem= d.get("mem",0)
        tmem= d.get("target_mem",0)
        tc= d.get("timecost",0)
        ttc= d.get("target_time",0)
        col= d.get("color","blue")
        title_txt= f"BFS={BFS_d}, mem={mem:.2f}->{tmem:.2f}, time={tc:.2f}->{ttc:.2f}"
        net_c.add_node(n, label=n, shape="dot", size=20, borderWidth=1,
                       title=title_txt, color={"border":"black","background":col})
    for (u,v) in subg.edges():
        c= subg[u][v].get("color","gray")
        net_c.add_edge(u,v, color=c)

    net_c.write_html("static/compgraph_layer.html", notebook=False, open_browser=False)
    return f'<h2>CompGraph BFS={layer_num}</h2><iframe src="/static/compgraph_layer.html" width="600" height="500"></iframe>'

@app.route("/glossary_hyper")
def glossary_hyper():
    global HYP_GRAPH
    if not HYP_GRAPH or HYP_GRAPH.number_of_nodes()==0:
        return "<h2>No Hypergraph found</h2>"

    module_desc= {
        "attention":"Multi-head self-attention mechanism",
        "ffn":"Feedforward network",
        "layernorm":"LayerNorm for stable activations",
        "dropout":"Regularization to prevent overfitting",
        "embedding":"Embedding layer",
        "pooler":"Pooling for final representation"
    }
    table_html= "<table border='1' cellpadding='6' cellspacing='0'><tr><th>Node</th><th>Description</th></tr>"
    for n in sorted(HYP_GRAPH.nodes()):
        desc="General model component"
        low_n= n.lower()
        for key in module_desc:
            if key in low_n:
                desc= module_desc[key]
                break
        table_html+= f"<tr><td>{n}</td><td>{desc}</td></tr>"
    table_html+="</table>"
    return f"<h2>Hypergraph Glossary</h2>{table_html}"

@app.route("/glossary_comp")
def glossary_comp():
    global COMP_GRAPH, COMP_SHAPE_INFO
    if not COMP_GRAPH or COMP_GRAPH.number_of_nodes()==0:
        return "<h2>No CompGraph found</h2>"

    table_html= "<table border='1' cellpadding='6' cellspacing='0'><tr>"
    table_html+= "<th>Node</th><th>Operation</th><th>BFS dist</th>"
    table_html+= "<th>Input shape</th><th>Output shape</th><th>GPU Mem (KB)</th>"
    table_html+="</tr>"

    for n,d in sorted(COMP_GRAPH.nodes(data=True)):
        bfs_d= d.get("BFS_distance","?")
        op_type= interpret_operation(n)
        in_shape,out_shape= COMP_SHAPE_INFO.get(n, ("?","?"))
        mem_kb= estimate_gpu_memory(n,out_shape)
        table_html += f"<tr><td>{n}</td><td>{op_type}</td><td>{bfs_d}</td>"
        table_html += f"<td>{in_shape}</td><td>{out_shape}</td><td>{mem_kb}</td></tr>"

    table_html+="</table>"
    return f"<h2>CompGraph Glossary</h2>{table_html}"

###############################################################################
# EXPORTS
###############################################################################

def graph_to_json(nx_graph):
    import networkx as nx
    data_out= {"nodes":[],"edges":[]}
    for n,d in nx_graph.nodes(data=True):
        item={"name":n}
        for k,v in d.items():
            item[k]=v
        data_out["nodes"].append(item)
    for s,t in nx_graph.edges():
        e_data= nx_graph[s][t]
        data_out["edges"].append({
            "source":s,
            "target":t,
            "color": e_data.get("color","gray")
        })
    return data_out

def graph_to_csv(nx_graph):
    import io
    output= io.StringIO()
    output.write("NODES\n")
    output.write("name,BFS_distance,mem,target_mem,timecost,target_time,color\n")
    for n,d in nx_graph.nodes(data=True):
        bfs= d.get("BFS_distance","")
        m  = d.get("mem","")
        tm = d.get("target_mem","")
        tc = d.get("timecost","")
        ttc= d.get("target_time","")
        col= d.get("color","")
        output.write(f"{n},{bfs},{m},{tm},{tc},{ttc},{col}\n")
    output.write("\nEDGES\n")
    output.write("source,target,color\n")
    for s,t in nx_graph.edges():
        e_data= nx_graph[s][t]
        c= e_data.get("color","gray")
        output.write(f"{s},{t},{c}\n")
    csv_str= output.getvalue()
    output.close()
    return csv_str

@app.route("/export_hypergraph")
def export_hypergraph_json():
    global HYP_GRAPH
    if not HYP_GRAPH:
        return jsonify({"error":"No hypergraph to export"})
    data_out= graph_to_json(HYP_GRAPH)
    return jsonify({"message":"Hypergraph exported","data":data_out})

@app.route("/export_hypergraph_csv")
def export_hypergraph_csv():
    global HYP_GRAPH
    if not HYP_GRAPH:
        return Response("No hypergraph to export",status=400)
    csv_str= graph_to_csv(HYP_GRAPH)
    return Response(csv_str,mimetype="text/csv",
                    headers={"Content-disposition":"attachment; filename=hypergraph_data.csv"})

@app.route("/export_comp_graph")
def export_comp_graph_json():
    global COMP_GRAPH
    if not COMP_GRAPH:
        return jsonify({"error":"No comp graph to export"})
    data_out= graph_to_json(COMP_GRAPH)
    return jsonify({"message":"CompGraph exported","data":data_out})

@app.route("/export_comp_graph_csv")
def export_comp_graph_csv():
    global COMP_GRAPH
    if not COMP_GRAPH:
        return Response("No comp graph to export",status=400)
    csv_str= graph_to_csv(COMP_GRAPH)
    return Response(csv_str,mimetype="text/csv",
                    headers={"Content-disposition":"attachment; filename=comp_graph_data.csv"})

@app.route("/export_q_table")
def export_q_table_api():
    global Q_TABLE
    data={}
    for s in Q_TABLE:
        data[str(s)] = Q_TABLE[s]
    return jsonify({"message":"Q-Table exported","table":data})

@app.route("/export_q_table_csv")
def export_q_table_csv():
    import io
    global Q_TABLE
    output= io.StringIO()
    output.write("state,a0,a1,a2,a3\n")
    for s in sorted(Q_TABLE.keys()):
        row= [str(s)] + [str(val) for val in Q_TABLE[s]]
        output.write(",".join(row)+"\n")
    csv_str= output.getvalue()
    output.close()
    return Response(csv_str,mimetype="text/csv",
                    headers={"Content-disposition":"attachment; filename=q_table.csv"})

@app.route("/export_comp_graph_layer/<layer_str>")
def export_comp_graph_layer_json(layer_str):
    global COMP_GRAPH
    if not COMP_GRAPH:
        return jsonify({"error":"No COMP_GRAPH. Run first!"})
    try:
        layer_num= int(layer_str)
    except:
        return jsonify({"error":f'Invalid layer number: {layer_str}'})
    subg= get_layer_subgraph(COMP_GRAPH, layer_num)
    if subg.number_of_nodes()==0:
        return jsonify({"error":f'No comp graph BFS_distance={layer_num}'})
    data_out= graph_to_json(subg)
    return jsonify({"message":f'CompGraph BFS layer={layer_num}',"data":data_out})

@app.route("/export_comp_graph_layer_csv/<layer_str>")
def export_comp_graph_layer_csv(layer_str):
    global COMP_GRAPH
    if not COMP_GRAPH:
        return Response("No COMP_GRAPH. Run first!",status=400)
    try:
        layer_num= int(layer_str)
    except:
        return Response(f"Invalid layer number: {layer_str}", status=400)
    subg= get_layer_subgraph(COMP_GRAPH, layer_num)
    if subg.number_of_nodes()==0:
        return Response(f"No compgraph BFS_distance={layer_num}", status=400)
    csv_str= graph_to_csv(subg)
    return Response(
        csv_str,
        mimetype="text/csv",
        headers={"Content-disposition":f"attachment; filename=compgraph_layer_{layer_num}.csv"}
    )

###############################################################################
# COLOR-CODING
###############################################################################

def mark_graph_changes(new_graph, old_graph):
    if not old_graph:
        for n in new_graph.nodes:
            new_graph.nodes[n]["color"]="green"
        for e in new_graph.edges:
            new_graph[e[0]][e[1]]["color"]="green"
        return

    old_nodes= old_graph.nodes(data=True)
    old_edges= set(old_graph.edges())

    for n,d in new_graph.nodes(data=True):
        if n not in old_graph:
            d["color"]="green"
        else:
            old_data= old_nodes[n]
            changed=False
            fields= ["BFS_distance","mem","timecost","target_mem","target_time"]
            for f in fields:
                old_val= old_data.get(f,None)
                new_val= d.get(f,None)
                if old_val!=new_val:
                    changed=True
                    break
            if changed:
                d["color"]="red"
            else:
                d["color"]="blue"

    new_edges= set(new_graph.edges())
    for e in new_edges:
        if e not in old_edges and (e[1],e[0]) not in old_edges:
            new_graph[e[0]][e[1]]["color"]="green"
        else:
            new_graph[e[0]][e[1]]["color"]="gray"

###############################################################################
# DYNAMIC MEM/TIME => CONVERGENCE
###############################################################################

def converge_mem_time():
    global COMP_GRAPH
    alpha=0.3
    if COMP_GRAPH:
        for n,d in COMP_GRAPH.nodes(data=True):
            if "mem" in d and "target_mem" in d:
                old_m= d["mem"]
                tgt_m= d["target_mem"]
                d["mem"]= old_m + alpha*(tgt_m- old_m)
            if "timecost" in d and "target_time" in d:
                old_t= d["timecost"]
                tgt_t= d["target_time"]
                d["timecost"]= old_t + alpha*(tgt_t- old_t)

def background_loop():
    while True:
        time.sleep(3)
        converge_mem_time()

bg_thread= threading.Thread(target=background_loop, daemon=True)
bg_thread.start()

###############################################################################
# Node Drag => update target_mem/time
###############################################################################

@app.route("/update_node_position", methods=["POST"])
def update_node_position():
    global COMP_GRAPH, COMP_SHAPE_INFO
    if not COMP_GRAPH:
        return jsonify({"error":"No comp graph"})

    data= request.get_json(force=True)
    node_id= data.get("node","")
    x_pos= float(data.get("x",0))
    y_pos= float(data.get("y",0))

    if node_id not in COMP_GRAPH:
        return jsonify({"error":f'Node {node_id} not in comp graph'})

    # BFS root
    root_candidates= [n for n in COMP_GRAPH.nodes if COMP_GRAPH.nodes[n].get("BFS_distance",0)==0]
    if not root_candidates:
        compute_bfs_levels(COMP_GRAPH)
        root_candidates= [n for n in COMP_GRAPH.nodes if COMP_GRAPH.nodes[n].get("BFS_distance",0)==0]
    root_node= root_candidates[0]

    COMP_GRAPH.nodes[node_id]["x"]= x_pos
    COMP_GRAPH.nodes[node_id]["y"]= y_pos

    rx= COMP_GRAPH.nodes[root_node].get("x",0)
    ry= COMP_GRAPH.nodes[root_node].get("y",0)
    dx= x_pos- rx
    dy= y_pos- ry
    dist= (dx*dx+ dy*dy)**0.5

    BFS_d= COMP_GRAPH.nodes[node_id].get("BFS_distance",0)

    # shape-based memory
    shape_inp, shape_out= COMP_SHAPE_INFO.get(node_id, ("?","?"))
    shape_mem= 0.0
    if isinstance(shape_out, tuple):
        numel=1
        for dim in shape_out:
            numel*= dim
        shape_mem= (numel*4)/1024  # in KB ?

    BFS_factor= BFS_d+1
    new_target_mem= shape_mem + BFS_factor * dist
    new_target_time= BFS_factor * dist * 2

    COMP_GRAPH.nodes[node_id]["target_mem"]= new_target_mem
    COMP_GRAPH.nodes[node_id]["target_time"]= new_target_time
    COMP_GRAPH.nodes[node_id]["geomDist"]= dist
    COMP_GRAPH.nodes[node_id]["color"]= "red"

    return jsonify({
        "message":"Node updated",
        "node": node_id,
        "BFS_d": BFS_d,
        "dist": dist,
        "shape_mem_kb": shape_mem,
        "target_mem": new_target_mem,
        "target_time": new_target_time
    })

###############################################################################
# Real-time Poll => memory/time
###############################################################################

@app.route("/poll_compgraph_data")
def poll_compgraph_data():
    """
    Returns BFS_distance, mem, timecost for each node.
    The front-end uses setInterval to poll & update tooltips every 3s
    """
    global COMP_GRAPH
    if not COMP_GRAPH:
        return jsonify({})
    out = {}
    for n,d in COMP_GRAPH.nodes(data=True):
        out[n] = {
            "BFS_distance": d.get("BFS_distance",0),
            "mem": round(d.get("mem",0), 2),
            "timecost": round(d.get("timecost",0), 2)
        }
    return jsonify(out)

###############################################################################
# MAIN PAGE
###############################################################################

@app.route("/", methods=["GET","POST"])
def single_page():
    global HYP_GRAPH, COMP_GRAPH, OLD_HYP_GRAPH, OLD_COMP_GRAPH
    global COMP_SHAPE_INFO, CONVERSATION, RL_CHOICE

    subgraph_html=""

    try:
        if request.method=="POST":
            model_label= request.form.get("model","BERT (base)")
            hf_id     = AVAILABLE_MODELS.get(model_label,"bert-base-uncased")
            gpu_sel   = request.form.get("gpu","T4")
            quant     = request.form.get("quant","None (FP32)")
            reasoning = request.form.get("reason","No reasoning")
            rl_algo   = request.form.get("rl_algo","No RL")
            RL_CHOICE = rl_algo
            user_txt  = request.form.get("textinput","").strip()
            act       = request.form.get("act","run")

            layer_input= request.form.get("layer_input","").strip()
            hyper_filter= request.form.get("hyper_filter","").strip()

            # NEW: sequence length
            seq_len_input= request.form.get("seq_len","").strip()
            try:
                seq_len= int(seq_len_input)
                if seq_len<1:
                    seq_len= 1
            except:
                seq_len= 32  # default

            model, tok = load_model(hf_id, quant)

            if rl_algo!="No RL":
                s= random.randint(0,9)
                a= random.randint(0,3)
                r= random.uniform(-1,1)
                update_q_table(s,a,r)
                CONVERSATION.append(f"RL({rl_algo}): Updated Q => s={s}, a={a}, r={r:.2f}")
            else:
                CONVERSATION.append("RL disabled (No RL).")

            if user_txt:
                CONVERSATION.append(f"User(Text): {user_txt}")

            if act=="run":
                new_hyp= extract_hypergraph(model, quant, reasoning)
                new_comp= extract_comp_graph(model, tok, gpu_sel, quant, reasoning)
                compute_bfs_levels(new_comp)
                mark_graph_changes(new_hyp, OLD_HYP_GRAPH)
                mark_graph_changes(new_comp, OLD_COMP_GRAPH)
                HYP_GRAPH= new_hyp
                COMP_GRAPH= new_comp
                OLD_HYP_GRAPH= new_hyp.copy()
                OLD_COMP_GRAPH= new_comp.copy()

                # shape trace
                shape_dict= trace_model_fx(model,tok)
                COMP_SHAPE_INFO.clear()
                for node_name,(inp,outp) in shape_dict.items():
                    nm= "COMP_"+ node_name
                    COMP_SHAPE_INFO[nm]= (inp,outp)

                scale_factor = GPU_MEMORY_SCALE.get(gpu_sel,1.0)
                # Assign initial distinct mem/time with new approximate_time_complexity
                for n,d in COMP_GRAPH.nodes(data=True):
                    BFS_d= d["BFS_distance"]
                    node_name= n.replace("COMP_","")

                    # shape-based memory
                    shape_in, shape_out= shape_dict.get(node_name, ("?","?"))
                    shape_mem= 0.0
                    if isinstance(shape_out, tuple):
                        numel=1
                        for dim in shape_out:
                            numel*= dim
                        shape_mem= (numel*4)/1024.0

                    # time complexity (just an integer cost)
                    node_time= approximate_time_complexity(node_name, reasoning, seq_len)

                    mem_init= (BFS_d+1)* shape_mem * scale_factor
                    time_init= (BFS_d+1)* node_time

                    d["mem"] = mem_init
                    d["target_mem"] = mem_init
                    d["timecost"] = time_init
                    d["target_time"] = time_init * 1.5  # e.g. so we have a separate target

                CONVERSATION.append(f"Action: run => BFS + shape + seq_len={seq_len} => assigned mem/time")

            elif act=="fine_tune":
                toy_finetune(model)
                new_hyp= extract_hypergraph(model, quant, reasoning)
                new_comp= extract_comp_graph(model, tok, gpu_sel, quant, reasoning)
                compute_bfs_levels(new_comp)
                mark_graph_changes(new_hyp,OLD_HYP_GRAPH)
                mark_graph_changes(new_comp,OLD_COMP_GRAPH)
                HYP_GRAPH= new_hyp
                COMP_GRAPH= new_comp
                OLD_HYP_GRAPH= new_hyp.copy()
                OLD_COMP_GRAPH= new_comp.copy()

                shape_dict= trace_model_fx(model,tok)
                COMP_SHAPE_INFO.clear()
                for node_name,(inp,outp) in shape_dict.items():
                    nm="COMP_"+ node_name
                    COMP_SHAPE_INFO[nm]=(inp,outp)

                scale_factor = GPU_MEMORY_SCALE.get(gpu_sel,1.0)
                for n,d in COMP_GRAPH.nodes(data=True):
                    BFS_d= d["BFS_distance"]
                    node_name= n.replace("COMP_","")
                    shape_in, shape_out= shape_dict.get(node_name, ("?","?"))
                    shape_mem=0
                    if isinstance(shape_out, tuple):
                        numel=1
                        for dim in shape_out:
                            numel*= dim
                        shape_mem= (numel*4)/1024.0
                    node_time= approximate_time_complexity(node_name, reasoning, seq_len)
                    mem_init= (BFS_d+1)* shape_mem* scale_factor
                    time_init= (BFS_d+1)* node_time
                    d["mem"]= mem_init
                    d["target_mem"]= mem_init
                    d["timecost"]= time_init
                    d["target_time"]= time_init*1.5

                CONVERSATION.append(f"Action: fine_tune => done + seq_len={seq_len} => mem/time assigned")

            elif act=="speak":
                new_hyp= extract_hypergraph(model, quant, reasoning)
                new_comp= extract_comp_graph(model, tok, gpu_sel, quant, reasoning)
                compute_bfs_levels(new_comp)
                mark_graph_changes(new_hyp,OLD_HYP_GRAPH)
                mark_graph_changes(new_comp,OLD_COMP_GRAPH)
                HYP_GRAPH= new_hyp
                COMP_GRAPH= new_comp
                OLD_HYP_GRAPH= new_hyp.copy()
                OLD_COMP_GRAPH= new_comp.copy()

                shape_dict= trace_model_fx(model,tok)
                COMP_SHAPE_INFO.clear()
                for node_name,(inp,outp) in shape_dict.items():
                    nm="COMP_"+node_name
                    COMP_SHAPE_INFO[nm]=(inp,outp)

                scale_factor = GPU_MEMORY_SCALE.get(gpu_sel,1.0)
                for n,d in COMP_GRAPH.nodes(data=True):
                    BFS_d= d["BFS_distance"]
                    node_name= n.replace("COMP_","")
                    shape_in, shape_out= shape_dict.get(node_name, ("?","?"))
                    shape_mem=0
                    if isinstance(shape_out, tuple):
                        numel=1
                        for dim in shape_out:
                            numel*= dim
                        shape_mem= (numel*4)/1024.0
                    node_time= approximate_time_complexity(node_name, reasoning, seq_len)
                    mem_init= (BFS_d+1)* shape_mem* scale_factor
                    time_init= (BFS_d+1)* node_time
                    d["mem"]= mem_init
                    d["target_mem"]= mem_init
                    d["timecost"]= time_init
                    d["target_time"]= time_init*1.5

                CONVERSATION.append(f"Action: speak => placeholder => seq_len={seq_len} => mem/time assigned")

            elif act=="layer_view":
                if COMP_GRAPH:
                    try:
                        ln= int(layer_input)
                        subg= get_layer_subgraph(COMP_GRAPH, ln)
                        if subg.number_of_nodes()==0:
                            subgraph_html= f"<h4>No compgraph BFS_distance={ln}</h4>"
                        else:
                            from pyvis.network import Network
                            net_c= Network(height='500px',width='600px',directed=False)
                            net_c.repulsion(node_distance=140,central_gravity=0.1)
                            for n,d in subg.nodes(data=True):
                                BFS_d= d.get("BFS_distance",0)
                                m= d.get("mem",0)
                                tm= d.get("target_mem",0)
                                t= d.get("timecost",0)
                                tt= d.get("target_time",0)
                                col= d.get("color","blue")
                                title_txt= f"BFS={BFS_d}, mem={m:.2f}->{tm:.2f}, time={t:.2f}->{tt:.2f}"
                                net_c.add_node(n,label=n,shape="dot",size=20,borderWidth=1,
                                               title=title_txt,
                                               color={"border":"black","background":col})
                            for (u,v) in subg.edges():
                                c= subg[u][v].get("color","gray")
                                net_c.add_edge(u,v,color=c)
                            net_c.write_html("static/layer_view.html", notebook=False, open_browser=False)
                            subgraph_html= f"<h3>CompGraph BFS layer={ln}</h3><iframe src='/static/layer_view.html' width='600' height='500'></iframe>"
                    except:
                        subgraph_html= f"<h4>Invalid BFS layer: {layer_input}</h4>"
                else:
                    subgraph_html= "<h4>No COMP_GRAPH available, please run first!</h4>"

            elif act=="hyper_filter_view":
                if HYP_GRAPH:
                    from pyvis.network import Network
                    subg= build_subgraph_hyper(HYP_GRAPH, hyper_filter)
                    if subg.number_of_nodes()==0:
                        subgraph_html= f"<h4>No hypergraph nodes match '{hyper_filter}'</h4>"
                    else:
                        net_h= Network(height='500px',width='600px',directed=False)
                        net_h.repulsion(node_distance=140,central_gravity=0.1)
                        for n,d in subg.nodes(data=True):
                            net_h.add_node(n,label=n,shape="dot",size=20,borderWidth=1,
                                           title=n,color={"border":"black","background":"blue"})
                        for (u,v) in subg.edges():
                            net_h.add_edge(u,v,color="gray")
                        net_h.write_html("static/hyper_filter_view.html",notebook=False,open_browser=False)
                        subgraph_html= f"<h3>Hypergraph filter='{hyper_filter}'</h3><iframe src='/static/hyper_filter_view.html' width='600' height='500'></iframe>"
                else:
                    subgraph_html= "<h4>No HYP_GRAPH available, please run first!</h4>"

        graph_html=""
        if HYP_GRAPH and COMP_GRAPH:
            from pyvis.network import Network
            if "BFS_distance" not in list(COMP_GRAPH.nodes(data=True))[0][1]:
                compute_bfs_levels(COMP_GRAPH)

            # Build hypergraph
            net_h = Network(height='500px', width='600px', directed=False)
            net_h.repulsion(node_distance=140, central_gravity=0.1)
            for n,d in HYP_GRAPH.nodes(data=True):
                col= d.get("color","blue")
                net_h.add_node(n,label=n,shape="dot",size=20,borderWidth=1,
                               title=n,color={"border":"black","background":col})
            for s,t in HYP_GRAPH.edges():
                c= HYP_GRAPH[s][t].get("color","gray")
                net_h.add_edge(s,t,color=c)
            net_h.write_html("static/hypergraph.html", notebook=False, open_browser=False)

            # Build compgraph
            net_c = Network(height='500px', width='600px', directed=False)
            net_c.repulsion(node_distance=140, central_gravity=0.1)
            for n,d in COMP_GRAPH.nodes(data=True):
                BFS_d= d.get("BFS_distance",0)
                m   = d.get("mem",0)
                tm  = d.get("target_mem",0)
                t   = d.get("timecost",0)
                tt  = d.get("target_time",0)
                col = d.get("color","blue")
                title_txt= f"BFS={BFS_d}, mem={m:.2f}->{tm:.2f}, time={t:.2f}->{tt:.2f}"
                net_c.add_node(n,label=n,shape="dot",size=20,borderWidth=1,
                               title=title_txt,color={"border":"black","background":col})
            for (u,v) in COMP_GRAPH.edges():
                e_col= COMP_GRAPH[u][v].get("color","gray")
                net_c.add_edge(u,v,color=e_col)
            net_c.write_html("static/compgraph.html", notebook=False, open_browser=False)

            # Insert the "drag" script + "poll" script for real-time updates
            with open("static/compgraph.html","r") as f:
                comp_html= f.read()

            drag_js= """
<script>
  var throttleTimeout = null;
  network.on("drag", function(params) {
    if (params.nodes.length > 0) {
      if (throttleTimeout) { return; }
      throttleTimeout = setTimeout(()=>{
        throttleTimeout=null;
        let nodeId= params.nodes[0];
        let nodePos= network.getPosition(nodeId);
        fetch("/update_node_position", {
          method:"POST",
          headers: {"Content-Type":"application/json"},
          body: JSON.stringify({"node":nodeId, "x":nodePos.x, "y":nodePos.y})
        })
        .then(resp=>resp.json())
        .then(dat=>{
          console.log("Dragging => new target mem/time:",dat);
          let BFS_d= dat.BFS_d;
          let dist= dat.dist.toFixed(2);
          let shapeM= dat.shape_mem_kb.toFixed(2);
          let mem= dat.target_mem.toFixed(2);
          let timeC= dat.target_time.toFixed(2);
          let newTitle= "BFS="+BFS_d+", dist="+dist+", shapeMem="+shapeM+" KB, mem->"+mem+", time->"+timeC;
          network.body.nodes[nodeId].options.title= newTitle;
          network.redraw();
        })
        .catch(err=>console.error(err));
      }, 150);
    }
  });
</script>
"""

            poll_js= """
<script>
  // Poll every 3s to get updated mem/time and update tooltips
  setInterval(()=>{
    fetch("/poll_compgraph_data")
    .then(r=>r.json())
    .then(dat=>{
       for(let nodeId in dat) {
         let BFS_d = dat[nodeId].BFS_distance;
         let mem   = dat[nodeId].mem.toFixed(2);
         let timeC = dat[nodeId].timecost.toFixed(2);
         // If the node is in the network, update tooltip
         if(network.body.nodes[nodeId]) {
            let newTitle= "BFS="+BFS_d+", mem="+mem+", time="+timeC;
            network.body.nodes[nodeId].options.title= newTitle;
         }
       }
       network.redraw();
    })
    .catch(err=>console.error(err));
  }, 3000);
</script>
"""

            comp_html= comp_html.replace("</body>", drag_js + "\n" + poll_js + "\n</body>")

            with open("static/compgraph.html","w") as f:
                f.write(comp_html)

            graph_html= f"""
<div style="margin:auto; width:90%; text-align:center;">
  <table style="margin:auto;"><tr>
     <td valign="top">
        <h2>Hypergraph</h2>
        <iframe src="/static/hypergraph.html" width="600" height="500"></iframe>
        <p><a href="/glossary_hyper" target="_blank">Hypergraph Glossary</a></p>
     </td>
     <td valign="top">
        <h2>Computational Graph</h2>
        <iframe src="/static/compgraph.html" width="600" height="500"></iframe>
        <p><a href="/glossary_comp" target="_blank">CompGraph Glossary</a></p>
        <p><i>Drag nodes to set new mem/time targets.<br>
        Memory/time also converge in the background over time,<br>
        so watch real-time updates in the node tooltips!</i></p>
     </td>
  </tr></table>
</div>

<div style="margin:auto; width:90%; text-align:center;">
  <h2>Q-Space Heatmap (2D)</h2>
  <img src="/q_space" alt="Q Heatmap"/>
  <br><a href="/q_space_interactive" target="_blank">3D Q-Space (click to open)</a>
  <p>You can rotate the 3D Q-Space by clicking/dragging in the plot.</p>
</div>

<div style="margin:auto; width:90%; text-align:center;">
  <p>
  <strong>Exports:</strong><br/>
  <a href="/export_hypergraph" target="_blank">Hypergraph JSON</a> |
  <a href="/export_hypergraph_csv" target="_blank">Hypergraph CSV</a><br/>
  <a href="/export_comp_graph" target="_blank">CompGraph JSON</a> |
  <a href="/export_comp_graph_csv" target="_blank">CompGraph CSV</a><br/>
  <a href="/export_q_table" target="_blank">Q-Table JSON</a> |
  <a href="/export_q_table_csv" target="_blank">Q-Table CSV</a>
  </p>
  <p>
    <strong>Subgraph routes</strong>:<br/>
    /layer_compgraph/0 => BFS=0 subgraph.<br/>
    /layer_hypergraph/attention => substring filter
  </p>
</div>
"""

        style_center= "style='text-align:center; margin:auto; width:90%;'"
        model_opts= "".join([f'<option value="{lbl}">{lbl}</option>' for lbl in AVAILABLE_MODELS])
        gpu_opts= "".join([f'<option value="{gp}">{gp}</option>' for gp in AVAILABLE_GPUS])
        quant_opts= "".join([f'<option value="{qq}">{qq}</option>' for qq in QUANT_METHODS])
        reas_opts= "".join([f'<option value="{rr}">{rr}</option>' for rr in REASONING_METHODS])
        rl_opts= "".join([f'<option value="{alg}">{alg}</option>' for alg in RL_ALGORITHMS])

        html= f"<html><head><title>AI Interpretability</title></head><body {style_center}>"
        html+= "<h1>AI Interpretability (Enhanced Time Complexity)</h1>"
        html+= "<form method='post'>"
        html+= "<label>Model:</label>"
        html+= f"<select name='model'>{model_opts}</select>"
        html+= "<label> GPU:</label>"
        html+= f"<select name='gpu'>{gpu_opts}</select>"
        html+= "<label> Quant:</label>"
        html+= f"<select name='quant'>{quant_opts}</select>"
        html+= "<label> Reasoning:</label>"
        html+= f"<select name='reason'>{reas_opts}</select>"
        html+= "<label> RL:</label>"
        html+= f"<select name='rl_algo'>{rl_opts}</select>"
        html+= "<br><br>"

        # new field for seq_len
        html+= "<label>Sequence Length (n): </label>"
        html+= '<input name="seq_len" placeholder="32" style="width:60px;"/>'
        html+= "<br><br>"

        html+= '<textarea name="textinput" rows="3" cols="60" placeholder="Type text here..."></textarea><br><br>'

        html+= "<button name='act' value='run'>Run</button> "
        html+= "<button name='act' value='fine_tune'>Fine-Tune</button> "
        html+= "<button name='act' value='speak'>Speak</button> "

        html+= "<hr><h3>View BFS Layer of CompGraph</h3>"
        html+= '(0 = input layer only)<br>'
        html+= '<input name="layer_input" placeholder="1" /> '
        html+= '<button name="act" value="layer_view">Show CompGraph Subgraph</button>'

        html+= "<hr><h3>Hypergraph Filter</h3>"
        html+= '<input name="hyper_filter" placeholder="attention or layer.1" /> '
        html+= '<button name="act" value="hyper_filter_view">Show Hypergraph Filter</button>'

        html+= "</form>"

        html+= "<h2>Conversation Log</h2>"
        for line in CONVERSATION:
            html+= f"<p>{line}</p>"
        html+= "<br>"

        html+= subgraph_html
        html+= graph_html

        html+= "</body></html>"
        return html

    except Exception as ex:
        tb= traceback.format_exc()
        CONVERSATION.append(f"[ERROR]: {tb}")
        return f"<h3>Internal Server Error</h3><pre>{tb}</pre>", 500

if __name__=="__main__":
    print("Starting Flask on port=8000, debug=False ...")
    app.run(host="0.0.0.0", port=8000, debug=False)
'''

# Write interpret_flask.py
with open("interpret_flask.py","w") as f:
    f.write(FULL_CODE)

print("✅ Wrote interpret_flask.py (Enhanced Time Complexity). Starting flask in background...")

# Start Flask in background
!nohup python interpret_flask.py > server.log 2>&1 &

print("⏳ Polling localhost:8000 for up to 30s to confirm server up...")

ready=False
for i in range(30):
    time.sleep(1)
    try:
        r = requests.get("http://127.0.0.1:8000")
        if r.status_code==200:
            print(f"✅ Flask responded after {i+1} sec. Proceeding.")
            ready=True
            break
    except:
        pass

if not ready:
    print("⚠️ Could not confirm flask is up after 30s. Attempting ngrok anyway...")

print("🔗 Start ngrok =>")
from pyngrok import ngrok

NGROK_AUTH_TOKEN = "2tedizGJDmEJMDtfhCR6ftx8EyX_2RaA7t8nPn17SU7jiyK1H"  # Replace with your token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(8000)
print("🌐 Public URL =>", public_url)

print("✅ Done. Check the logs if needed =>")
time.sleep(2)
!tail -n 20 server.log


🔴 Killing anything on port=8000 + old ngrok sessions.
^C
📦 Installing dependencies (quietly)...
✅ Wrote interpret_flask.py (Enhanced Time Complexity). Starting flask in background...
⏳ Polling localhost:8000 for up to 30s to confirm server up...
✅ Flask responded after 5 sec. Proceeding.
🔗 Start ngrok =>
🌐 Public URL => NgrokTunnel: "https://a744-34-87-155-94.ngrok-free.app" -> "http://localhost:8000"
✅ Done. Check the logs if needed =>
Starting Flask on port=8000, debug=False ...
 * Serving Flask app 'interpret_flask'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
Press CTRL+C to quit
127.0.0.1 - - [05/Mar/2025 03:43:22] "GET / HTTP/1.1" 200 -
